In [20]:
# Import declarations
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from rfpimp_MC import *
import category_encoders as ce
from pandas.api.types import is_string_dtype, is_object_dtype, is_categorical_dtype

df_raw = pd.read_csv("../data collection/XboxOne_GameSales.csv", encoding='latin-1')

In [21]:
# Normalize String function from section 7.4 of the text book
# Used to normalize missing string data types to np.nan
# In this case 'N/A' and empty
def df_normalize_strings(df):
    for col in df.columns:
        if is_string_dtype(df[col]) or is_object_dtype(df[col]):
            df[col] = df[col].str.lower()
            df[col] = df[col].fillna(np.nan)
            df[col] = df[col].replace('unknown', np.nan)

df_normalize_strings(df_raw)

In [22]:
def df_string_to_cat(df):
    for col in df.columns:
        if is_string_dtype(df[col]) or is_object_dtype(df[col]):
            df[col] = df[col].astype('category').cat.as_ordered()

def df_cat_to_catcode(df):
    for col in df.columns:
        if is_categorical_dtype(df[col]):
            df[col] = df[col].cat.codes + 1

df_string_to_cat(df_raw)
df_cat_to_catcode(df_raw)

C:\Users\sjare\AppData\Local\Temp\ipykernel_17860\2494755687.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if is_categorical_dtype(df[col]):
C:\Users\sjare\AppData\Local\Temp\ipykernel_17860\2494755687.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if is_categorical_dtype(df[col]):
C:\Users\sjare\AppData\Local\Temp\ipykernel_17860\2494755687.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if is_categorical_dtype(df[col]):
C:\Users\sjare\AppData\Local\Temp\ipykernel_17860\2494755687.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if is_categorical_dtype(df[col]):


In [23]:
def fix_missing_num(df, colname):
    df[colname+'_na'] = pd.isnull(df[colname])
    df[colname] = df[colname].replace('N/A', np.nan)
    df[colname].fillna(df[colname].median(), inplace=True)

fix_missing_num(df_raw, 'Year')

C:\Users\sjare\AppData\Local\Temp\ipykernel_17860\1123911679.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[colname].fillna(df[colname].median(), inplace=True)


In [24]:
# Removing games where global sales are 0. Meaning 0 sold
df = df_raw[(df_raw.Global != 0)]

# Dropping features related to the target
df.drop(['North America', 'Europe', 'Japan', 'Rest of World'], axis= 1, inplace=True)

# Dropping Position in order as it will throw off my prediction, since they are all in order
df.drop('Pos', axis=True, inplace=True)

# Dropping Game, since all are unique values and won't help predictability
df.drop('Game', axis=True, inplace=True)

# Dropping Game, since all are unique values and won't help predictability
df.drop(['Year','Year_na'], axis=True, inplace=True)

C:\Users\sjare\AppData\Local\Temp\ipykernel_17860\2310641355.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['North America', 'Europe', 'Japan', 'Rest of World'], axis= 1, inplace=True)
C:\Users\sjare\AppData\Local\Temp\ipykernel_17860\2310641355.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('Pos', axis=True, inplace=True)
C:\Users\sjare\AppData\Local\Temp\ipykernel_17860\2310641355.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [25]:
# Test Function from text book from Chapter 6
def test(X, y, n_estimators=150):
    rf = RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1, oob_score=True)
    rf.fit(X, y)
    oob = rf.oob_score_
    n = rfnnodes(rf)
    h = np.median(rfmaxdepths(rf))
    print(f"OOB R^2 {oob:.5f} using {n:,d} tree nodes with {h} median tree height")
    return rf, oob

# Creating model
X = df.drop('Global', axis=1)
y = df['Global']

# Creating Model AND testing 10x
for i in range(10):
    rf, oob_initial = test(X, y) 

OOB R^2 0.34685 using 43,372 tree nodes with 15.0 median tree height
OOB R^2 0.33542 using 43,508 tree nodes with 14.5 median tree height
OOB R^2 0.33229 using 43,360 tree nodes with 15.0 median tree height
OOB R^2 0.33166 using 43,392 tree nodes with 15.0 median tree height
OOB R^2 0.34501 using 43,278 tree nodes with 15.0 median tree height
OOB R^2 0.32358 using 43,440 tree nodes with 15.0 median tree height
OOB R^2 0.32952 using 43,186 tree nodes with 15.0 median tree height
OOB R^2 0.33333 using 43,368 tree nodes with 15.0 median tree height
OOB R^2 0.33053 using 43,380 tree nodes with 15.0 median tree height
OOB R^2 0.33853 using 43,198 tree nodes with 15.0 median tree height
